In [1]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-
#!/usr/bin/env python
import pickle as pickle
import glob, os, inspect
import numpy as np
import argparse
import time
import seaborn as sns
import matplotlib
matplotlib.use("Qt5Agg")
#matplotlib.use('Agg')
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import pandas as pd
import sys,inspect
import utils
from utils import *
import sys
sys.path.append('../Methods/Models/Utils')
import global_utils

In [2]:
# sysname, tidx = "newmodel1", 2

# current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# results_dir = os.path.dirname(current_dir) + "/Results"
# print(results_dir)

# eval_path = os.path.join(results_dir, '{}/Evaluation_Data'.format(sysname))
# print(eval_path)

# model_path = os.path.join(results_dir, '{}/Trained_Models'.format(sysname))
# print(model_path)

# fig_path = os.path.join(results_dir, '{}/Eval_Figures'.format(sysname))
# if os.path.isdir(fig_path) == False:
#     os.mkdir(fig_path)

sysname, tidx = "newmodel1", 2
file_path = os.path.abspath("plot_newmodel1.ipynb")
current_directory = os.path.dirname(file_path)
parent_directory = os.path.dirname(current_directory)

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

#current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
results_dir = os.path.join(os.path.dirname(current_directory), "Results")  # Use os.path.join for path concatenation
print(results_dir)

eval_path = os.path.join(results_dir, sysname, 'Evaluation_Data')
print(eval_path)

model_path = os.path.join(results_dir, sysname, 'Trained_Models')
print(model_path)

fig_path = os.path.join(results_dir, sysname, 'Eval_Figures')
if not os.path.isdir(fig_path):
    os.makedirs(fig_path)  # Use os.makedirs to create parent directories if needed
print(fig_path)

Current Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Postprocess
Parent Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\newmodel1\Evaluation_Data
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\newmodel1\Trained_Models
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\newmodel1\Eval_Figures


In [4]:
maxLyp = 1.0
dt = 1
trained_models = None

In [3]:
models = [[os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_400-DL_40-Nqr_5-A_0.0-J_2.0-fJ_0-V_5-NL_0-IPL_20-IUL_0-REG_1e-07-AU_0-NICS_100"),"HQR-5,V-5,K=400,T=40,n_tests=100"]]
samples = ["HQR-5,V-5,K=400,T=40,n_tests=100"]
title = "Sea Surface Temperatures"

In [5]:
rmse_dict = dict()
vpt_dict = dict()
targets = dict()
outputs = dict()
sp_outputs = dict()
sp_targets = dict()
Wouts = dict()
coeffs = dict()

for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'results.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            qs = QResults()
            qs.rmnse_avg_test = rs['rmnse_avg_TEST']
            qs.rmnse_avg_train = rs['rmnse_avg_TRAIN']
            qs.n_pred_005_avg_test = rs['num_accurate_pred_005_avg_TEST']
            qs.n_pred_005_avg_train = rs['num_accurate_pred_005_avg_TRAIN']
            qs.n_pred_050_avg_test = rs['num_accurate_pred_050_avg_TEST']
            qs.n_pred_050_avg_train = rs['num_accurate_pred_050_avg_TRAIN']
            qs.model_name = rs['model_name']
            #if qs.rmnse_avg_test != np.inf and qs.rmnse_avg_train != np.inf:
                #print(rs.keys())
            #print(qs.model_name)
            #print('train={}, test={}'.format(qs.rmnse_avg_train, qs.rmnse_avg_test))
            #qs.info()

            pred_test = rs['predictions_all_TEST']
            truth_test = rs['truths_all_TEST']
            M = len(pred_test)
            print('{} Number of test'.format(qs.model_name), M)
            rmsels = []
            vpts = []
            for j in range(M):
                    rmsels.append(calNRMSE(pred_test[j], truth_test[j]))
                    vpts.append(calVPT(pred_test[j], truth_test[j],dt=dt, maxLyp=maxLyp))##

            rmse_dict[label] = np.mean(np.array(rmsels), axis=0)
            print(rmse_dict[label].shape)

            vpt_dict[label] = np.array(vpts)

            targets[label] = truth_test[99]
            outputs[label] = pred_test[99]

            # For frequency
            sp_outputs[label] = rs['sp_pred_TEST']
            sp_targets[label] = rs['sp_true_TEST']
    else:
        print('Not found {}'.format(fname))

    if trained_models is not None:
        rfolder, label = trained_models[i][0], trained_models[i][1]
        fname = os.path.join(rfolder, 'data.pickle')
        if os.path.isfile(fname):
            print('File existed: ', fname)
            with open(fname, 'rb') as rfile:
                #try:
                rs = pickle.load(rfile)
                #except:
                #    continue
                print(rs.keys())
                coeffs[label] = np.array(rs['coeffs'])
                Wouts[label]  = rs['W_out'][:-1].reshape((coeffs[label].shape[0], -1))
                print(Wouts[label].shape, coeffs[label].shape)
        else:
            print('Not found saved model {}'.format(fname))

hqrc_pinv-RDIM_5-N_used_400-DL_40-Nqr_5-A_0.0-J_2.0-fJ_0-V_5-NL_0-IPL_20-IUL_0-REG_1e-07-AU_0-NICS_100 Number of test 100
(20,)


In [6]:
def normalize_array(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

In [7]:
# PLOTTING
cmap = plt.get_cmap("RdBu")
ecmap = plt.get_cmap("summer_r")
#plt.style.use('seaborn-colorblind')
plt.rc('font', family='serif')
plt.rc('mathtext', fontset='cm')
plt.rcParams['font.size']=9
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [8]:
label = samples[0]
temp_target = targets[label]
temp_out = outputs[label]
num_dimensions = temp_target.shape[1]
# Create subplots
fig, axs = plt.subplots(num_dimensions, 1, figsize=(10, 10))
# Iterate over each dimension
for dim in range(num_dimensions):
    # Normalize target and output arrays for current dimension
    target_dim = temp_target[:, dim]
    out_dim = temp_out[:, dim]
    
    norm_target_dim = normalize_array(target_dim)
    norm_out_dim = normalize_array(out_dim)
    
    # Plot target and output arrays
    axs[dim].plot(norm_target_dim, label='Target')
    axs[dim].plot(norm_out_dim, label='Prediction')
    
    # Set y-axis limits to [0, 1]
    axs[dim].set_ylim(0, 1)
    
    # Add labels and legend
    axs[dim].set_xlabel('Time')
    axs[dim].set_ylabel(f'Dimension {dim+1}')
    axs[dim].legend()

# Add title
plt.suptitle('Target and Prediction for Each Dimension')
# Adjust layout
plt.tight_layout()

# Plot frequency
plt.show()

In [31]:
from mpl_toolkits.mplot3d import Axes3D

# Assuming temp_target and temp_out have shape (20, 5)

# Extract the first three dimensions
target_3d = temp_target[:, :3]
out_3d = temp_out[:, :3]

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the first three dimensions of temp_target and temp_out
ax.plot(target_3d[:, 0], target_3d[:, 1], target_3d[:, 2], label='Target')
ax.plot(out_3d[:, 0], out_3d[:, 1], out_3d[:, 2], label='Prediction')

# Set labels and legend
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Dimension 3')
ax.legend()

# Set plot title
plt.title('First Three Dimensions: Target vs Prediction')

# Show plot
plt.show()


In [ ]:

# outbase = 'tidx_{}_{}_v2'.format(tidx, title)
# outbase = os.path.join(fig_path, outbase)
# for ftype in ['pdf', 'svg', 'png']:
#     transparent = (ftype != 'png')
#     plt.savefig('{}_{}_rs4.{}'.format(outbase, sysname, ftype), bbox_inches='tight', transparent=transparent, dpi=600)

# plt.show()